In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import quad,nquad
from scipy.optimize import minimize
import scipy.sparse as ssp
import scipy as sp
from scipy.special import gamma

import time

%matplotlib inline

In [ ]:
lat_const = 155

red_area = 8/np.sqrt(3)*np.pi**2/lat_const**2 # volume of reduced zone in rec. lattice.
sl = 4*np.pi/np.sqrt(3)/lat_const # side length of parallelogram reduced zone.
print(red_area)
lb = -sl/2 #left bound
rb = sl/2 #right bound

psi = np.array([0])#np.array([np.pi/180])
V = np.array([0.00036])
#V = np.array([0.00000001])

m_el = 1
m_h = 1 
epsilon_r = 1

d = 0


Coulomb_prefactor = - 2*np.pi/epsilon_r


### !!! this n determines an upper bound for how many bands we will have in our calculations in the other parts too.
n_bands_calc = 3
bands_calc = (2*n_bands_calc+1)**2 #this is for calculating the Bloch states. If this is not large then the bands are just wrong.


n = n_bands_calc-0
bands_mat = (2*n+1)**2 


N = 1
L = 1/(2*np.pi)**2
U1_prefactor = L
D = L*N
K = -D*2*np.pi/epsilon_r

q = np.array([0,0])

order_mat = np.array([10])  #order to build matrix for Riesz.

In [ ]:
pos = int(np.floor((2*n+1)**2/2))

indexes = np.arange(-pos,pos+1) 
indindexes = indexes[:,None,None] + indexes[None,:,None] - indexes[None,None,:]
indindexes = indindexes + pos
indindexes[indindexes>=(2*n+1)**2] = -1
indindexes[indindexes<0] = -1

interesting_pos = np.array(np.where(indindexes!=-1))


thing = np.arange(-n, n+1)
i, j = np.meshgrid(thing, thing)
ih = 4*np.pi/np.sqrt(3)/lat_const*i
jh = 4*np.pi/np.sqrt(3)/lat_const*j
Gs = np.stack((ih,jh),axis=-1) 
Gs = Gs.reshape(-1,Gs.shape[-1])

In [ ]:
def basis_norm_1D(k):
    assert k.shape[0] == 2
    return np.sqrt(k[0]**2 + k[1]**2 - k[0]*k[1])

In [ ]:
def get_bloch(k_list,n,V,psi,m):
        
    time_start = time.time()
    
    eigvals_stor = np.zeros((k_list.shape[0],(2*n+1)**2),dtype=complex)
    eigvecs_stor = np.zeros((k_list.shape[0],(2*n+1)**2,(2*n+1)**2),dtype=complex)

    for nn in range(k_list.shape[0]):
        k_init = k_list[nn]
        
        kinetic = 1/2 * np.diag(basis_norm_1D((Gs + k_init).T)**2)/m

        V1 = V*np.exp(1.0j*psi)
        V2 = np.conj(V1)

        pot = np.zeros(kinetic.shape,dtype=complex)
        dim = kinetic.shape[0]

        thing = np.ones(dim)
        thing1 = np.diag(V1*thing)
        thing2 = np.diag(V2*thing)

        pi = V2*np.ones((2*n+1)**2-1)
        pi[2*n::2*n+1] = 0
        pi = np.diag(pi,1)

        mi = V1*np.ones((2*n+1)**2-1)
        mi[2*n::2*n+1] = 0
        mi = np.diag(mi,-1)

        pj = V2*np.ones((2*n+1)**2-(2*n+1))
        pj = np.diag(pj,2*n+1)

        mj = V1*np.ones((2*n+1)**2-(2*n+1))
        mj = np.diag(mj,-(2*n+1))

        pij = V1*np.ones((2*n+1)**2-(2*n+2))
        pij[2*n::2*n+1] = 0
        pij = np.diag(pij,(2*n+2))

        mij = V2*np.ones((2*n+1)**2-(2*n+2))
        mij[2*n::2*n+1] = 0
        mij = np.diag(mij,-(2*n+2))

        pot = pi + mi + pj + mj + pij + mij

        mat = kinetic + pot

        zres = np.linalg.eig(mat)

        if m > 0:
            sort_help = np.argsort(np.real(zres[0]))
        if m < 0:
            sort_help = np.argsort(-np.real(zres[0]))
        eigvals = zres[0][sort_help]
        eigvecs = zres[1][:,sort_help]
        
        # when testing with the print function in the line below, I found out that the first two vectors or so are only almost eigenvectors (most entries the same, but not all). But this should not be an issue.
        #print(np.dot(mat,eigvecs[:,2])/eigvecs[:,2])
        eigvals_stor[nn] = eigvals
        eigvecs_stor[nn] = eigvecs/np.linalg.norm(eigvecs,axis=0)

    time_stop = time.time()
    #print("Runtime:",time_stop-time_start)  
    
    return eigvals_stor,eigvecs_stor

In [ ]:
def get_bands(order_mat,bands_mat,n,n_bands_calc):

    nodes_mat, weights_mat = np.polynomial.legendre.leggauss(order_mat)
    xx,yy = np.meshgrid(nodes_mat,nodes_mat)
    nodes_mat = np.stack((xx,yy), axis=-1) 
    nodes_mat = nodes_mat.reshape(-1,nodes_mat.shape[-1])
    nodes_mat = 1/2*(1-nodes_mat)*lb+1/2*(nodes_mat+1)*rb  
    
    weights_mat = 1/2*(rb-lb)*weights_mat
    xx,yy = np.meshgrid(weights_mat,weights_mat)
    weights_mat = np.sin(2*np.pi/3)*xx*yy
    weights_mat = weights_mat.reshape(-1) 
    print(sum(weights_mat))
    
    eigval_el_mat,eigvec_el_mat = get_bloch(nodes_mat+q,n_bands_calc,V,psi,m_el) 

    eigval_h_mat,eigvec_h_mat = get_bloch(-nodes_mat,n_bands_calc,V,psi,m_h)
    

    
    eigval_el_mat = eigval_el_mat[:,:bands_mat]
    eigvec_el_mat = eigvec_el_mat[:,:,:bands_mat]
    
    eigval_h_mat = eigval_h_mat[:,:bands_mat]
    eigvec_h_mat = eigvec_h_mat[:,:,:bands_mat]

    
    return nodes_mat,weights_mat,eigvec_el_mat,eigvec_h_mat,eigval_el_mat,eigval_h_mat

In [ ]:
len_test = len(order_mat)

nodes_mats = []
weights_mats = []
eigvec_el_mats = []
eigvec_h_mats = []
eigval_el_mats = []
eigval_h_mats = []


for i in range(len_test):       
    n_mat,w_mat,ec_el_mat,ec_h_mat,el_el_mat,el_h_mat = get_bands(order_mat[i],bands_mat,n,n_bands_calc)
    nodes_mats.append(n_mat)
    weights_mats.append(w_mat)
    eigvec_el_mats.append(ec_el_mat)
    eigvec_h_mats.append(ec_h_mat)
    eigval_el_mats.append(el_el_mat)
    eigval_h_mats.append(el_h_mat)

In [ ]:
def basis_norm_bc(k):
    #
    return np.sqrt(k[:,:,0]**2 + k[:,:,1]**2 - k[:,:,0]*k[:,:,1])

In [ ]:
def screened_coulomb(k_i,k_j):
    #
    broadcasted = k_i - k_j + Gs[:,None,:] - Gs[None,:,:]
    #
    return Coulomb_prefactor * np.exp(-d*basis_norm_bc(broadcasted))/basis_norm_bc(broadcasted)

In [ ]:
# GUER := get U element regular
def GUER(egvecs_e,egvecs_h,i,j,nodes,apply_mask):
    
    e_i = np.conj(egvecs_e[i,:,0])
    h_i = np.conj(egvecs_h[i,:,0])
    
    e_j = egvecs_e[j,:,0]
    h_j = egvecs_h[j,:,0]
    
    time1 = time.time()
    
    fa4 = h_j[indindexes]
    
    fa4[indindexes<0] = 0
    
    k_i = np.copy(nodes[i])
    k_j = np.copy(nodes[j])
    
    fa_coul = screened_coulomb(k_i,k_j)

    if apply_mask == True:
        np.fill_diagonal(fa_coul,0)
    time2 = time.time()
    #print("Time for prep:", time2-time1)
    
    time1 = time.time()
    
    somma = e_i[:,None,None] * h_i[None,:,None] * e_j[None,None,:] * fa4 * fa_coul[:,None,:]
    somma = np.sum(somma)
    time2 = time.time()
    #print("Time for somma:", time2-time1)
        
    return U1_prefactor * somma
    

In [ ]:
def phi(n,i):
    return basis_norm_1D(n[i])**2/(basis_norm_1D(n[i])**2 + basis_norm_1D((n[i]-n).T)**2) * 1/basis_norm_1D((n[i] - n).T)

In [ ]:
def phi_helper(k1,k2):
    return basis_norm_1D(k1)**2/(basis_norm_1D(k1)**2 + basis_norm_1D((k1-k2).T)**2) * 1/basis_norm_1D((k1 - k2).T)

In [ ]:
def variational_principle(egvecs_e,egvecs_h,egvals_e,egvals_h,nodes,weights):
    
    dim = egvals_e.shape[0]
    
    U_sing = np.zeros((dim,))
    
    for i in range(dim):
        phi_eval = phi(nodes,i)
        phi_eval[i] = 0
        U_sing[i] = - np.sum(weights * phi_eval) + (np.pi**2 * basis_norm_1D(nodes[i]) - not_infty[i])
 
    U_sing = K*U_sing
    U_sing = np.diag(U_sing)
    
    
    time1 = time.time()
    U_reg = np.zeros((dim,dim))
    
    for i in range(dim):
        if np.mod(i,10) == 0:
            print(i)
        for j in range(dim):
            if i != j:
                U_reg[i,j] = weights[j] * GUER(egvecs_e,egvecs_h,i,j,nodes,False)
            else:
                U_reg[i,j] = weights[j] * GUER(egvecs_e,egvecs_h,i,j,nodes,True)
     
    time2 = time.time()
    #print("Time for U_reg:", time2-time1)
    

    H1 = np.diag(egvals_e[:,0])
    H2 = np.diag(egvals_h[:,0])
    
    H_tot = H1 + H2 + U_reg + U_sing
    
    #print(abs(H_tot)-H_tot)
    
    results = np.linalg.eig(H_tot)

    sort_help = np.argsort(np.real(results[0]))
    eigvals = results[0][sort_help]
    eigvecs = results[1][:,sort_help]

    print(eigvals[:-1])
    

    return np.amin(eigvals),H1,H2,H_tot,eigvals

In [ ]:
nodes = nodes_mats[0]
dim = nodes.shape[0]

limit=200

not_infty = np.zeros(dim)

for i in range(dim):
    #print(i)
    term1,_ = quad(lambda y: quad(lambda x: phi_helper(nodes[i], np.array([x,y])),rb,limit)[0],-limit,limit)
    term2,_ = quad(lambda y: quad(lambda x: phi_helper(nodes[i], np.array([x,y])),-limit,lb)[0],-limit,limit)
    term3,_ = quad(lambda y: quad(lambda x: phi_helper(nodes[i], np.array([x,y])),lb,rb)[0],rb,limit)
    term4,_ = quad(lambda y: quad(lambda x: phi_helper(nodes[i], np.array([x,y])),lb,rb)[0],-limit,lb)
    not_infty[i] =  np.sin(2*np.pi/3)*(term1 + term2 + term3 + term4)
    #print(not_infty[i])
    

In [ ]:
eigval_el_mat = eigval_el_mats[0]
eigval_h_mat = eigval_h_mats[0]
eigvec_el_mat = eigvec_el_mats[0]
eigvec_h_mat = eigvec_h_mats[0]
nodes_mat = nodes_mats[0]
weights_mat = weights_mats[0]

result,H1,H2,H_tot,eigvals = variational_principle(eigvec_el_mat,eigvec_h_mat,eigval_el_mat,eigval_h_mat,nodes_mat,weights_mat)

In [ ]:
###
# Code for plotting the band structure.
###

e = eigval_el_mats[0]
h = eigval_h_mats[0]



nod = nodes_mats[0]
np.set_printoptions(threshold=np.inf)

idx = int(np.sqrt(e.shape[0]))
n = int(e.shape[0]/2)
m = int(idx/2)


g = nod[n+m]
kp = nod[-m]
k = nod[m]
#kpp = np.array([ 0.02276878,  0.01303966])
kpp = np.array([0.02340311, 0.01363739])


tol = 0.01

compare = np.linspace(-0.02,0.02,100)

helper = nod[np.abs(nod[:,1]/nod[:,0]-1/np.sqrt(3))<tol]
#print(helper)
'''
plt.figure()
plt.scatter(helper[:,0],helper[:,1])
plt.plot(compare[:],compare[:])
plt.plot(compare,np.zeros(len(compare)))
plt.show()
'''
print(helper[np.argmax(np.linalg.norm(helper,axis=1))])

tol = 0.0001
def points_on_line(points, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    m =0
    b =0
    if x2 - x1 == 0:
        is_vertical = True
    else:
        m = (y2 - y1) / (x2 - x1)
        b = y1 - m * x1
        is_vertical = False
    
    # Function to check if a point is on the line
    def is_on_line(x, y):
        if is_vertical:
            return np.abs(x-x1) < tol
        else:
            return np.abs(y - m * x - b) < tol

    index = np.zeros(points.shape[0])
    for i in range(len(index)):
        index[i] = is_on_line(points[i,0],points[i,1])
    boolean_mask = index.astype(bool)
    return points[boolean_mask],m,b,boolean_mask

p1 = kpp
p2 = k
res,m,b,boolean_mask = points_on_line(nod,p1,p2)
loc = np.array([p1,p2])
'''
plt.figure()
plt.plot(np.linalg.norm(res-res[0,:],axis=1),e[boolean_mask,:4])
plt.show()

plt.figure()
#plt.plot(np.array([-0.0219,-0.0183]),np.array([-0.0233,-0.0233]))
#plt.scatter(fp[:,0], m*fp[:,0] + b)
plt.scatter(res[:,0],res[:,1])
plt.scatter(loc[:,0],loc[:,1])
plt.show()

'''
wv = 40
xlist = []
ylist = []
lfl = []
for band_idx in np.arange(wv):
    schedule = np.array([k,kp,kpp,k])
    band_energy = np.array([])
    xaxis = np.array([])
    for i in range(0,3):
        p1 = schedule[i]
        p2 = schedule[i+1]
        loc = np.array([p1,p2])
        res,m,b,boolean_mask = points_on_line(nod,p1,p2)
        if i > 0:
            xloc = np.linalg.norm(res-p1,axis=1) + np.amax(xaxis)#+np.linalg.norm(schedule[i]-schedule[i-1])
        else:
            xloc = np.linalg.norm(res-p1,axis=1) + np.linalg.norm(schedule[i]-schedule[i-1])

        #print(xaxis,xloc)
        eloc = e[boolean_mask,band_idx]

        xaxis = np.append(xaxis,xloc)
        band_energy = np.append(band_energy,eloc)

        plt.figure()

        #plt.scatter(res[:,0],res[:,1])
        #plt.scatter(loc[:,0],loc[:,1])
        #plt.show()
        #print(np.amax(xloc))

        #plt.figure()
        #plt.scatter(xaxis,band_energy,s=3)
        #plt.show()
    xlist.append(xaxis)
    ylist.append(1000*27.2*band_energy)

for band_idx in np.arange(wv):
    schedule = np.array([k,kp,kpp,k])
    band_energy = np.array([])
    xaxis = np.array([])
    for i in range(0,3):
        p1 = schedule[i]
        p2 = schedule[i+1]
        loc = np.array([p1,p2])
        res,m,b,boolean_mask = points_on_line(nod,p1,p2)
        if i > 0:
            xloc = np.linalg.norm(res-p1,axis=1) + np.amax(xaxis)#+np.linalg.norm(schedule[i]-schedule[i-1])
        else:
            xloc = np.linalg.norm(res-p1,axis=1) + np.linalg.norm(schedule[i]-schedule[i-1])

        #print(xaxis,xloc)
        eloc = h[boolean_mask,band_idx]

        xaxis = np.append(xaxis,xloc)
        band_energy = np.append(band_energy,eloc)

        plt.figure()

        #plt.scatter(res[:,0],res[:,1])
        #plt.scatter(loc[:,0],loc[:,1])
        #plt.show()
        #print(np.amax(xloc))
        if i == 2:
            print(np.amax(xaxis))
        #plt.figure()
        #plt.scatter(xaxis,band_energy,s=3)
        #plt.show()
    xlist.append(xaxis)
    ylist.append(1000*27.2*band_energy)

print(np.amin(np.abs(ylist[1]-ylist[0])))
print(np.amin(np.abs(ylist[wv+1]-ylist[wv])))
import matplotlib.pyplot as plt
figsize_a4_inches = (8.27, 11.69)
plt.figure(figsize = figsize_a4_inches,dpi=1000)

plt.ylabel("E [meV]",fontsize=16)

lfl = np.array([0.0234,0.0468,0.07205])

#plt.vlines(lfl,-15,100,linewidth=0.9,zorder=0,color="black")


for i in range(wv):
    xaxis = xlist[i]
    yaxis = ylist[i]
    plt.scatter(xaxis,yaxis,s=1,color="blue")
    
    
    
vline_positions = [0,0.0234,0.0468,0.07205,0.1158015022]
labels = ["$\kappa$","$\gamma$","$\kappa'$","$\kappa''$","$\kappa$"]

for position in vline_positions:
    plt.axvline(x=position, color='black',zorder=0)  

plt.xticks(vline_positions, labels,fontsize=16)

plt.xlim(0,0.1158015022)
plt.ylim(-15,100)
    
plt.tight_layout()  

plt.show()

figsize_a4_inches = (8.27, 11.69)
plt.figure(figsize = figsize_a4_inches,dpi=1000)

plt.ylabel("E [meV]",fontsize=16)
lfl = np.array([0.0234,0.0468,0.07205])

#.vlines(lfl,-100,30,linewidth=0.9,zorder=0,color="black")



for i in range(wv,2*wv):
    xaxis = xlist[i]
    yaxis = ylist[i]
    plt.scatter(xaxis,yaxis,s=1,color="blue")

    
vline_positions = [0,0.0234,0.0468,0.07205,0.1158015022]
labels = ["$\kappa$","$\gamma$","$\kappa'$","$\kappa''$","$\kappa$"]

for position in vline_positions:
    plt.axvline(x=position, color='black',zorder=0)  

plt.xticks(vline_positions, labels,fontsize=16)

plt.xlim(0,0.1158015022)
plt.ylim(-100,30)
plt.tight_layout()  

plt.show()